<a href="https://colab.research.google.com/github/fakabbir/AI-Playbook/blob/master/Glove_%2B_GCN_%2B_CoreNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading the vector file
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-04-19 05:17:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-19 05:17:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-19 05:17:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [22]:
# Checking cuda version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [23]:
# Downloading the right cude version pytorch
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu111

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu111


In [24]:
# Checking files
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data


In [ ]:
# Converting into vector space
import os
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [26]:
embeddings_index['dancing']

array([ 8.8927e-01,  1.3668e-01,  1.0018e-01,  8.6889e-02, -3.7379e-04,
        1.2549e+00, -3.0605e-01,  8.2326e-01, -3.1645e-01, -2.6105e-01,
        8.8670e-02, -3.7473e-01,  2.3649e-01,  1.4771e-01, -3.5600e-01,
        1.3845e-01, -1.6199e-01,  4.6586e-01,  2.3392e-01, -3.7496e-01,
       -1.4306e-01,  1.6502e-01,  7.2604e-01, -5.7993e-01,  2.1166e-01,
        6.6273e-01, -7.3482e-01, -2.5190e-01,  4.5720e-01, -4.2118e-02,
       -8.2452e-01, -2.2596e-01,  4.6759e-01, -1.9811e-01,  5.5191e-01,
       -4.3812e-01, -4.5375e-02,  2.4324e-01, -2.0800e-01, -6.9571e-01,
        1.8216e-01,  3.7280e-01, -6.7986e-01,  1.0618e-01,  2.1276e-02,
       -2.1635e-01, -1.1486e+00, -8.0755e-03,  1.2753e-01, -6.2336e-01,
       -6.1599e-01, -3.0707e-01,  9.8371e-03,  9.4887e-01, -2.1619e-01,
       -2.0712e+00, -1.1367e-01,  7.9654e-01,  2.3070e-01,  7.2508e-01,
        3.6047e-01,  1.2606e+00,  6.0255e-03,  1.5010e-01, -2.1836e-01,
        7.9186e-01,  6.6293e-01, -7.5083e-01,  1.0351e-01, -2.03

In [30]:
from torch import nn
import torch

In [31]:
# Prepare embedding matrix
num_tokens = 1 + len(embeddings_index)
embedding_dim = len(embeddings_index['the'])
embedding_matrix = np.zeros((num_tokens, embedding_dim))

embedding_matrix_idx = {}
for idx,word in enumerate(embeddings_index):
    embedding_matrix[1+idx] = embeddings_index[word]
    embedding_matrix_idx[word] = 1 +idx

weight = torch.FloatTensor(embedding_matrix)
embedding = nn.Embedding.from_pretrained(weight)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [48]:
from torch import nn

class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        params = {
            'hidden_size': 192,
            'input_size': embedding_dim,
            'bias': True,
            'dropout': 0.2,
            'bidirectional': True }

        model = nn.LSTM(**params)
        self.embedding = embedding
        self.model = model
        self.fc = nn.Linear(192, 2)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        self.embedding(text)
        return self.fc(embedded)

In [49]:
model_sentiment = SentimentClassifier()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [54]:
def predict(text):
    with torch.no_grad():
        text = torch.tensor(embedding_matrix_idx[text])
        output = model_sentiment(text)
        return output

In [55]:
predict('cats')

RuntimeError: ignored

In [56]:
torch.tensor(embedding_matrix_idx["cat"])

tensor(5451)